In [1]:
%load_ext autoreload 
%autoreload 2

import xarray as xr
import numpy as np
import os
import pandas as pd
from glob import glob
from denseweight import DenseWeight
import tensorflow as tf
from keras import backend as K

2024-02-04 07:39:22.289930: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 07:39:24.931139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 07:39:38.175956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Our goal is to weight individual data points based on the rarity of their target values. 

## source https://link.springer.com/article/10.1007/s10994-021-06023-5

In [3]:
# First let's create a normalized distribution for the observations
if region_name == 'CONUS':
    obs = xr.open_dataset('Data/GLEAM/RZSM_anomaly.nc').sel(time = slice('2000-01-01','2015-12-31'))
elif region_name == 'australia':
    obs = xr.open_dataset('Data_australia/GLEAM/RZSM_anomaly.nc').sel(time = slice('2000-01-01','2015-12-31'))

#Now compute the min max standardization (because this is what we are using as our prediction with deep learning model
min_= obs.min(dim='time')
max_ = obs.max(dim='time')

#Now min max
stand = (obs - min_) / (max_-min_).load()

In [3]:
#small subset
sub = stand.RZSM[:,10,10].values
# Define DenseWeight
dw = DenseWeight(alpha=1.0)
# Fit DenseWeight and get the weights for the 1000 samples
weights = dw.fit(sub)

In [5]:
max_idx = [idx for idx,i in enumerate(sub) if i ==np.nanmax(sub)] #Maximum value
weights[max_idx]

array([3.63728292])

In [6]:
min_idx = [idx for idx,i in enumerate(sub) if i ==np.nanmin(sub)] #Maximum value
weights[min_idx]

array([3.655096])

In [7]:
#This shows that the most common values have the lowest weight
min_weight_idx = [idx for idx,i in enumerate(weights) if i ==np.nanmin(weights)] #Maximum value
sub[min_weight_idx]

array([0.45902889, 0.45891538, 0.45892979, 0.45892236, 0.45898656,
       0.45893287, 0.45888535, 0.45898887])

In [7]:
# now apply the function to every grid cell
#We want to save the original data into the first index in the last channel and the new kde into the 2nd index in last channel
outKDE = np.empty(shape=(obs.time.shape[0], obs.latitude.shape[0],obs.longitude.shape[0],2))
outKDE.shape

(5844, 48, 96, 2)

# Alpha value of 1.0 doesn't seem to be helping much with learning. It may be too sensitive.

## Let's try value of 0.8 for alpha

In [8]:
alpha=0.5

In [9]:
for idx,i in enumerate(range(obs.latitude.shape[0])):
    for jdx,j in enumerate(range(obs.longitude.shape[0])):
        if np.count_nonzero(np.isnan(stand.RZSM[:,idx,jdx].values)) > 0:
            outKDE[:,idx,jdx,0] = 0
            outKDE[:,idx,jdx,1] = 0
        else:
            # Define DenseWeight
            dw = DenseWeight(alpha=alpha)
            # Fit DenseWeight and get the weights for the number of time samples
            outKDE[:,idx,jdx,0] = stand.RZSM[:,idx,jdx].values
            outKDE[:,idx,jdx,1] = dw.fit(stand.RZSM[:,idx,jdx].values)
    
            #Now sort the indices based only on the kde values
            sorted_indices = np.argsort(outKDE[:, idx,jdx, 0])
            outKDE[:,idx,jdx,0] = outKDE[sorted_indices,idx,jdx,0]
            outKDE[:,idx,jdx,1] = outKDE[sorted_indices,idx,jdx,1]

In [10]:
#Now save the data for later use with loss function
save_dir = 'Data/model_npy_inputs/weighted_density'
os.system(f'mkdir -p {save_dir}')
np.save(f'{save_dir}/weighted_density_{region_name}_{alpha}.npy',outKDE)

In [139]:
#Now test with the loss functions
y_pred = np.load('Data/model_npy_inputs/Wk_0_EX_input_data/EX10_RZSM_testing_input.npy')[:,:,:,-1]
y_true = np.load('Data/model_npy_inputs/Verification_data/OBS_RZSM_GLEAM_lead_0_test_masked.npy')


In [5]:
y_pred.shape

(1144, 48, 96)

In [6]:
y_true.shape

(1144, 48, 96)

In [7]:
y_pred = y_pred[0:11,:,:]
y_true = y_true[0:11,:,:]

In [8]:
y_pred.shape

(11, 48, 96)

In [130]:
## Source https://github.com/yingkaisha/keras-unet-collection/blob/main/keras_unet_collection/losses.py
def _crps_tf_dense(y_true, y_pred, factor=0.08):
    
    '''
    core of (pseudo) CRPS loss.
    
    y_true: two-dimensional arrays
    y_pred: two-dimensional arrays
    factor: importance of std term
    '''
    def mae_by_whole_region_mean(y_true, y_pred, factor=0.08):
        #MAE by grid cell
        mae = tf.cast(K.mean(tf.abs(y_pred - y_true)),dtype=tf.float32)
        dist = tf.cast(np.nanmean(np.nanstd(y_pred,axis=0)),dtype=tf.float32)
        crps_exp = mae - factor*dist
    
        #Just provide the names so we know what they are
        norm_obs = density[:,:,:,0] #Shape is (5844, 48, 96)
        norm_kde = density[:,:,:,1] #Shape is (5844, 48, 96)
    
        #y_pred shape is TensorShape([11, 48, 96])
        y_pred_mean = tf.experimental.numpy.nanmean(y_pred,axis=0)
        #Try with just taking the mean of the predictions
        #Find the absolute value difference which is equal to us finding which observation value is closest to the prediction value (y_pred)
        # y_pred shape is (48,96)
        abs_diff = tf.math.abs(norm_obs - tf.experimental.numpy.nanmean(y_pred,axis=0)) # Shape is TensorShape([5844, 48, 96])
        
        #Find the date index that is the lowest
        min_day_indices = tf.math.argmin(abs_diff,axis=0) #Shape is (48,96)
    
        #Check some values (looks good)
        # x,y=10,20
        # min_arg = argmin[x,y]
        # abs_diff
        # norm_obs[min_arg,x,y]
        # y_pred_mean[x,y]
    
        #Now we need to subset norm_kde to find its value
        # Use tf.gather to obtain the subset of 'norm_kde'
        subset_array = tf.gather(norm_kde, min_day_indices[:, tf.newaxis, tf.newaxis], axis=0)
        # Squeeze the tensor to remove singleton dimensions
        avg_kde = K.mean(tf.squeeze(subset_array, axis=[1, 2]))
    
    
        
        #Now multiply the grid with the data and take the mean to produce a scalar
        dense_crps = crps_exp * avg_kde

        # #check the other result where we apply the mean first to produce a scalar with no dense crps
        # mae = tf.cast(K.mean(tf.abs(y_pred - y_true)),dtype=tf.float32)
        # #Then find another scalar
        # dist = np.nanmean(np.nanstd(y_pred,axis=0))
        # dist = tf.cast(dist,dtype=tf.float32)
        # #Then compute a scalar
        # (mae - factor*dist)
        
        return(dense_crps)



    def mae_by_grid_cell_then_take_mean(y_true, y_pred, factor=0.08):
        #right now this is too difficult to try and get to work performance wise, So it's currently unfinished. 
        #MAE by grid cell
        mae = tf.abs(y_pred - y_true)
        dist = np.nanstd(y_pred,axis=0)
        crps_exp = mae - factor*dist
    
        #Just provide the names so we know what they are
        norm_obs = density[:,:,:,0] #Shape is (5844, 48, 96)
        norm_kde = density[:,:,:,1] #Shape is (5844, 48, 96)
    
        

        #Find the absolute value difference which is equal to us finding which observation value is closest to the prediction value (y_pred)
        # y_pred shape is (48,96)
        abs_diff = tf.math.abs(norm_obs -y_pred[:,tf.newaxis,:,:]) # Shape is TensorShape([11, 5844, 48, 96])
        
        #Find the date index that is the lowest
        min_day_indices = tf.math.argmin(abs_diff,axis=1) #Shape is (11,48,96)
    
        #Check some values (looks good)
        # x,y=10,20
        # min_arg = argmin[x,y]
        # abs_diff
        # norm_obs[min_arg,x,y]
        # y_pred_mean = tf.experimental.numpy.nanmean(y_pred,axis=0) #y_pred shape is TensorShape([48, 96])
        # y_pred_mean[x,y]
    
        #Now we need to subset norm_kde to find its value
        # Use tf.gather to obtain the subset of 'norm_kde'
        subset_array = tf.gather(norm_kde, min_day_indices[tf.newaxis, :], axis=0)
        subset_array.shape
        # Squeeze the tensor to remove singleton dimensions
        avg_kde = K.mean(tf.squeeze(subset_array, axis=[1, 2]))

        #Now multiply the grid with the data and take the mean to produce a scalar
        dense_crps = crps_exp * avg_kde
        return(dense_crps)

    return mae_by_whole_region_mean(y_true, y_pred, factor=0.08)

def crps2d_tf_dense(y_true, y_pred, factor=0.08):
    
    '''
    (Experimental)
    An approximated continuous ranked probability score (CRPS) loss function:
    
        CRPS = mean_abs_err - factor * std
        
    * Note that the "real CRPS" = mean_abs_err - mean_pairwise_abs_diff
    
     Replacing mean pairwise absolute difference by standard deviation offers
     a complexity reduction from O(N^2) to O(N*logN) 
    
    ** factor > 0.1 may yield negative loss values.
    
    Compatible with high-level Keras training methods
    
    Input
    ----------
        y_true: training target with shape=(batch_num, x, y, 1)
        y_pred: a forward pass with shape=(batch_num, x, y, 1)
        factor: relative importance of standard deviation term.
        
    '''
    # tf.print(f'y_true shape = {y_true.shape}')
    # tf.print(f'y_pred shape = {y_pred.shape}')
    
    y_pred = tf.squeeze(tf.convert_to_tensor(y_pred))
    y_true = tf.squeeze(tf.cast(y_true, y_pred.dtype))

    #For testing
    # y_pred = y_pred[0:66,:,:]
    # y_true = y_true[0:66,:,:]
    
    # tf.print(y_true.shape)
    # tf.print(y_pred.shape)
    # batch_num = y_pred.shape.as_list()[0]
    
    crps_out = 0
    start_,end_ = 0,11

    #testing
    # factor=0.08
    
    #Split the batch sizes to not get an Nan at the last part of the batch
    new_range = y_true.shape[0]//11
    for i in range(new_range):
        # print(i)
        # break
        crps_out += _crps_tf_dense(y_true=y_true[start_:end_, ...], y_pred =y_pred[start_:end_, ...], factor=factor)
        #must add 11 each time
        start_ +=11
        end_ += 11
    
    #Divide by total number of inits
    # crps_out = crps_out/new_range

    #Have them all as a sum
    crps_out = crps_out
        
    return crps_out

global density
density = np.load('Data/model_npy_inputs/weighted_density/weighted_density.npy')
density = tf.cast(np.nan_to_num(density, nan=0),dtype=tf.float32)

In [ ]:
weights

In [ ]:
len(sub)

In [ ]:
len(weights)

In [ ]:
np.max(weights)

In [ ]:
np.min(weights)

In [ ]:
#Load sample data
test_ref = np.load('Data/model_npy_inputs/Wk_0_EX_input_data/EX10_RZSM_testing_input.npy')
obs = np.load(

In [ ]:
# #check loss values
# def open_loss(experiment):
#     loss = pd.read_csv(f'Losses_with_OBS/Wk_1/Wk1_{experiment}')
#     print(loss)


In [ ]:
# open_loss('EX12')

In [ ]:
# # #Load model and check summary
# model = load_model(f'checkpoints/Wk_1/Wk1_EX5',compile=False) #don't need the custom loss function for predictions
# model.summary() 


In [ ]:
# # Iterate through the layers and print the shape of each layer's weights
# num
# for layer in m.layers:
#     if layer.weights:
#         print(f"Layer: {layer.name}")
#         for weight in layer.weights:
#             print(f"  {weight.name}: {weight.shape}")

In [ ]:
lead

In [ ]:
experiment_name

In [ ]:
m.weights()

In [ ]:
model.load_weights(m)

In [ ]:
m.summary()